In [ ]:
from PIL import Image, ImageChops, ImageEnhance
import os
import itertools

def convert_to_ela_image(path, quality=75):
    """Standard ELA with quality=75 - used in 99% of real papers"""
    original = Image.open(path).convert('RGB')
    
    # Save with compression
    temp_path = 'temp_ela.jpg'
    original.save(temp_path, 'JPEG', quality=quality)
    compressed = Image.open(temp_path)
    
    # Compute difference
    ela_image = ImageChops.difference(original, compressed)
    
    # Get extrema for scaling
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    # Enhance and scale properly
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
    # Clean up temp file
    if os.path.exists(temp_path):
        os.remove(temp_path)
    
    return ela_image

In [ ]:
real_image_path = '/kaggle/input/200k-real-vs-ai-visuals-by-mbilal/my_real_vs_ai_dataset/my_real_vs_ai_dataset/real/00012.jpg'
Image.open(real_image_path)

In [ ]:
convert_to_ela_image(real_image_path, 85)

In [ ]:
dataset_path = '/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2'

# Paths
real_path = os.path.join(dataset_path, 'Au')
fake_path = os.path.join(dataset_path, 'Tp')

image_size = (128, 128)  # Smaller = faster training, still works well
X = []
Y = []

In [ ]:
print("Loading real images...")
for filename in tqdm(os.listdir(real_path)[:4000]):  # limit for speed, remove [:4000] for full
    if filename.endswith(('jpg', 'jpeg', 'png', 'tif', 'tiff')):
        full_path = os.path.join(real_path, filename)
        try:
            ela = convert_to_ela_image(full_path, 75)
            ela = ela.resize(image_size)
            ela = np.array(ela) / 255.0  # Normalize!
            X.append(ela)
            Y.append(0)  # Real = 0
        except:
            continue

In [ ]:
print("Loading fake images...")
for filename in tqdm(os.listdir(fake_path)[:4000]):
    if filename.endswith(('jpg', 'jpeg', 'png', 'tif', 'tiff')):
        full_path = os.path.join(fake_path, filename)
        try:
            ela = convert_to_ela_image(full_path, 75)
            ela = ela.resize(image_size)
            ela = np.array(ela) / 255.0
            X.append(ela)
            Y.append(1)  # Fake = 1
        except:
            continue

In [ ]:
# Convert to numpy
X = np.array(X)
Y = to_categorical(Y, 2)

print(f"Dataset shape: {X.shape}, Labels: {Y.shape}")